In [ ]:
###--------------------------------------------- 環境設定 --------------------------------------------------------------###

#     その日初めて実行する時、またはしばらく使用せずランタイムがリセットされた時は必ず実行してください。　　　　　　#

#-------------------------------------------------------------------------------------------------------------------------------#

%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

############################################ ユーザー設定 ###########################################
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
###　　　　　　　　　　　　　　　　　　　　　　　　　　　                                 　　　　　　　　　　　　購入株の情報　　　　　　　　　　　　　　　                       　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　###
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

brand_number = ""                      # 購入する株の銘柄の数字を"と"の間に入力
stock_number = ""                      # 購入する株数を"と"の間に入力

#------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
###　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　                                  　　　　アカウント情報　                      　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　###
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

myID = ""                              # ユーザーIDを"と"の間に入力
myPassword = ""                       # パスワードを"と"の間に入力
pIDnumber = ""                        # 暗証番号を"と"の間に入力

#------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
#################################################################################################



#------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
###　　　　　　　　　　                                  　　　　　　　　　　　　　　　　GMOクリック証券のURLアドレス　　　　　　　　                      　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　###
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

url = "https://sec-sso.click-sec.com/loginweb/" 

#------------------------------------------------------------------------------------------------------------------------------------------------------------------------#


#------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
###　　　　　　　　　　　　　　　　                                 　　　　　　　　　　　　　　　　　　　　メインプログラム　                       　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　###
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

# ブラウザをヘッドレスモードで使用するための設定
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.headless = True
driver_path = "/usr/bin/chromedriver"                       # chromedriverがある場所
service = Service(executable_path=driver_path)               
driver = webdriver.Chrome(service=service, options=options)

# GMOクリック証券のログインページへアクセス
driver.get(url)
driver.maximize_window()
print(driver.title)                                             # アクセスしたサイトを表示
time.sleep(1)                                                # 短時間アクセスによる負荷軽減

# ログイン
userID = driver.find_element(By.NAME, "j_username")        # ユーザーIDを入力する場所を検索
password = driver.find_element(By.NAME, "j_password")     # パスワードを入力する場所を検索
login = driver.find_element(By.NAME, "LoginForm")          # ログインボタンを検索
userID.send_keys(myID)                                      # ユーザーID入力
password.send_keys(myPassword)                           # パスワード入力
login.click()                                                  # ログインボタンをクリック
# cur_url1 = driver.current_url
# print(cur_url1)                                             # 移動先のURL確認用

# 株式ページに移動
tag_id = driver.find_element(By.LINK_TEXT, "株式")          # 株式ボタンを検索
tag_id.click()                                                 # 株式ボタンをクリック
# cur_url2 = driver.current_url
# print(cur_url2)                                             # 移動先のURL確認用

# 銘柄検索
brand_id = driver.find_element(By.ID, "searchKey")           # 銘柄の検索
search_id = driver.find_element(By.ID, "meigaraSearchButton") # 検索ボタンを検索
if brand_number=="":                                        # 銘柄のナンバーを入力していない時の対策
  print ("銘柄が指定されていません。")
  driver.quit()
else:
  pass
brand_id.send_keys(brand_number)                           # 銘柄の入力
search_id.click()                                              # 検索ボタンをクリック
# cur_url3 = driver.current_url
# print(cur_url3)                                             # 移動先のURL確認用

# 注文
# ヘッドレスモードを再設定
element1 = driver.find_element(By.XPATH, "//input[@id='torihikiToggleMukigen']")  # 『一般信用』を検索
driver.execute_script('arguments[0].click();', element1)                             # 『一般信用』を選択

element2 = driver.find_element(By.XPATH, "//input[@id='baibaiToggleUri']")       # 売買区分の『売』を検索
driver.execute_script('arguments[0].click();', element2)                            # 売買区分を『売』に指定

stock_num_id = driver.find_element(By.ID, "jyuchuuSuuryo")                       # 取引数量の入力場所を検索
if stock_number=="":                                                              # 取引数量を入力していない時の対策
  print ("『取引数量』が指定されていません。")
  driver.quit()
else:
  pass
stock_num_id.send_keys(stock_number)                                           # 取引数量の入力

element3 = driver.find_element(By.XPATH, "//input[@id='marketOrder']")          # 『成行』ボックスを検索
driver.execute_script('arguments[0].click();', element3)                            # 『成行』ボックスにチェック

pIDnum_id = driver.find_element(By.ID, "pinCode")                                 # 取引暗証番号を入力する場所を検索
pIDnum_id.send_keys(pIDnumber)                                                  # 取引暗証番号の入力

element4 = driver.find_element(By.XPATH, "//input[@id='omitConfirm'][@type='checkbox']")  # 『確認省略』ボックスを検索
driver.execute_script('arguments[0].click();', element4)                             # 『確認省略』ボックスにチェック

# 注文されるまで繰り返し実行
while driver.find_element(By.ID, "orderButton"):
  confirm_id = driver.find_element(By.ID, "orderButton")
  driver.execute_script('arguments[0].click();', confirm_id)
  time.sleep(3)

  element1 = driver.find_element(By.ID, "orderPanelCompleteArea")
  element2 = driver.find_element(By.ID, "messageBlock")
  element3 = driver.find_element(By.XPATH, "//div[@id='messageBlock']/div[1]")

  # 注文が確定したため終了
  if element1.is_displayed():
    print("注文しました。")
    break

  # 在庫がないため在庫が発生するまで繰り返し    
  elif element2.is_displayed() and element3.get_attribute("class") == "m-alert-02 u-mt12":
    print("在庫がありません。")
    time.sleep(5)                                                                  # 短時間でのアクセスによるサイトへの負荷軽減処置
    continue
    print("再度注文します。")

  # 想定以外のことが発生しているため注文中止
  else:
    print("イレギュラー発生。中止します。")
    break

driver.quit()                                                                        # プログラム終了
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------#